# Visualization and statistical analysis

- Inclusive count: cells positive in any of the channel combination, then it is positive for this combination of channels.
- Exclusive count: cells positive in all of the channel combination, then it is positive for this combination of channels and is associated to this category.

Save the results as a 'destination folder/*-stats.csv' file

Define the result folder

In [ ]:
import yaml
from pathlib import Path
from ipyfilechooser import FileChooser

srcdir, dstdir = "", ""
if Path("config.yml").exists():
    with open("config.yml", "r") as file:
        config = yaml.safe_load(file)
        if "source" in config.keys():
            srcdir = Path(config["source"])
        if "destination" in config.keys():
            dstdir = Path(config["destination"])

fc = FileChooser(dstdir, select_desc="Destination")
display(fc)

In [ ]:
import pandas as pd

dstdir = Path(fc.selected) if fc.selected is not None else Path(dstdir)
filelistname = dstdir / "filelist.csv"
filelist = pd.read_csv(filelistname)
filelist

In [ ]:
from itertools import chain, combinations
from functools import partial
from functools import reduce
import operator
import numpy as np
import matplotlib.pyplot as plt
import tifffile
import napari
import dask


def get_files(dstdir, row, key=None):
    if key == "ims":
        return Path(row["folder"]) / row["name"]
    elif key == "regions":
        return Path(dstdir / str(row["name"]).replace(".ims", "-regions.json"))
    elif key == "labels":
        return Path(dstdir / str(row["name"]).replace(".ims", "-labels.tif"))
    elif key == "measurements":
        return Path(dstdir / str(row["name"]).replace(".ims", "-measurements.csv"))
    elif key == "stats":
        return Path(dstdir / str(row["name"]).replace(".ims", "-stats.csv"))
    else:
        return {
            "ims": get_files(dstdir, row, "ims"),
            "regions": get_files(dstdir, row, "regions"),
            "labels": get_files(dstdir, row, "labels"),
            "measurements": get_files(dstdir, row, "measurements"),
        }


def get_measurement_channels(df):
    """List the channels name from the measurement data"""
    return df.columns[6:]


def create_heatmaps(labels, df):
    channel_columns = [f"c{k}" for k in range(10) if f"c{k}" in df.columns]

    heatmaps = np.zeros([len(channel_columns), *labels.shape])
    for row in df.iloc:
        for k, c in enumerate(channel_columns):
            heatmaps[k][labels == row["label"]] = row[c]
    return heatmaps


def madstd(x):
    """Median std"""
    return 1.48 * np.median(np.abs(x - np.median(x)))


def powerset(iterable):
    """Compute the powerset of iterable"""
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))


def encode(row, channels, thresholds, encoder):
    """compute a code from the channels columns and the tresholds"""
    t = tuple([c for c in channels if row[c] > thresholds[c]])
    return encoder[t]


def encode_channels_exclusive(df, columns, thresholds):
    """Encode the channel in the data frame based on intensity

    Parameters
    ----------
    df : pd.DataFrame
    columns : List[str]
        list of columns names
    thresholds: List[float]
        list of threshold values

    Returns
    -------
    code_df and encoder, decoder dicts
    """
    pset = [x for x in powerset(columns)]
    encoder = {x: k for k, x in enumerate(pset)}
    decoder = {k: x for k, x in enumerate(pset)}
    decoder[0] = ("none",)
    decode_str = {
        a: b
        for a, b in enumerate(["+".join([str(e) for e in k]) for k in encoder.keys()])
    }
    cid = df.apply(
        partial(encode, channels=columns, encoder=encoder, thresholds=thresholds),
        axis=1,
    )
    code_df = pd.DataFrame({"label": df["label"], "id": cid})
    return code_df, decoder


def prod(iterable):
    return reduce(operator.mul, iterable, 1)


def encode_channels_inclusive(df, columns, thresholds):
    """Encode the channel in the data frame based on intensity

    Parameters
    ----------
    df : pd.DataFrame
    channels : List[str]
        list of channel names
    thresholds: List[float]
        list of threshold values

    Returns
    -------

    code_df and encoder, decoder dicts
    """
    pset = [x for x in powerset(columns)]
    decoder = {k + 1: "+".join([str(e) for e in x]) for k, x in enumerate(pset[1:])}
    code_df = pd.DataFrame(
        {
            "+".join([str(e) for e in k]): prod([df[e] > thresholds[e] for e in k])
            for k in pset[1:]
        }
    )
    code_df["label"] = df["label"]
    return code_df, decoder

def create_class_image(labels, codes, decoder, chunk_size=20):
    """Create a map of the binary codes as a label map

    Parameter
    ---------
    labels: np.array

    codes: pd.DataFrame
        data frame with a column label and a label code

    decoder: dict
        dictionary: {id: tuple(combination columns names)}

    Note add 1 to the code so that it is not set to background
    """
    def helper(stack, labels, codes, decoder):
        for row in codes.iloc:
            stack[labels == row["label"]] = row["id"] + 1
    
    stack = np.zeros(labels.shape, dtype=np.uint8)

    tsk = [
        dask.delayed(helper)(
            stack, labels, codes.iloc[n : n + chunk_size], decoder
        )
        for n in range(0, codes.shape[0], chunk_size)
    ]
    
    dask.compute(tsk)

    features = pd.DataFrame(
        {"code": ["background", *[" + ".join(decoder[k]) for k in decoder]]}
    )
    return stack, features

def create_class_masks(labels, codes, decoder, chunk_size=20):
    """Create a set of maps for each combination of labels

    Parameter
    ---------
    labels: np.array

    codes: pd.DataFrame
        data frame with a column label and a column per combination

    decoder: dict
        dictionary: {id: tuple(combination columns names)}

    """
    nc = len(decoder)
    stack = np.zeros([nc, *labels.shape], dtype=np.uint8)

    def helper(stack, labels, codes, decoder):
        for row in codes.iloc:
            for c in decoder:
                if row[decoder[c]] == 1:
                    stack[c - 1][labels == row["label"]] = 255

    tsk = [
        dask.delayed(helper)(
            stack, labels, codes.iloc[n : n + chunk_size], decoder
        )
        for n in range(0, codes.shape[0], chunk_size)
    ]

    dask.compute(tsk)
    names = [decoder[k] for k in decoder]
    return stack, names


def aggregate_combinations(input, decoder):
    """Aggregate the inputs based on combinations in the decoder

    Parameters
    ----------
    input: pd.DataFrame or np.array
        input on which to compute the aggregation
    decoder: dict
        mapping between keys of the input and the corresponding set of channels
    """
    output = input.copy()
    for k1 in decoder:
        for k2 in decoder:
            if len(decoder[k2]) > len(decoder[k1]):
                for y1 in decoder[k1]:
                    if y1 in decoder[k2]:
                        output[k1] = output[k1] + input[k2]
    return output

def map_intensity(labels, df, channels, chunk_size=20):
    """Set each label with the intensity store in the dataframe df

    The purpose of this is to help ind the adequate threshold

    Parameters
    ----------
    labels: ndarray (D,H,W)
        labels of the segmented nuclei in 3D
    df: pd.dataframe
        data frame with intensity measurement and matching labels indices
    channels: List
        list of the channels
    chunk_size: int
        size of the chunck

    Return
    ------
    map : ndarray (C,D,H,W)
        intensity map for each label
    """
    
    def helper(dst, labels, df, channels):
        for row in df.iloc:
            idx = labels == row["label"]
            for k, ch in enumerate(channels):
                dst[k][idx] = row[ch]

    dst = np.zeros((len(channels), *labels.shape))
    tsk = [
        dask.delayed(helper)(
            dst, labels, df.iloc[n : n + chunk_size], channels
        )
        for n in range(0, df.shape[0], chunk_size)
    ]
    dask.compute(tsk)

    return dst

In [ ]:
import ipywidgets as widgets

w = widgets.Dropdown(
    options=[(x, k) for k, x in enumerate(filelist["name"])],
    value=1,
    description="Image:",
)
display(w)

Load the image

In [ ]:
from imaris_ims_file_reader.ims import ims

row = filelist.iloc[w.value]
resolution_level = 1  # need to be the same than the one used for processing
img = ims(get_files(dstdir, row, "ims"), ResolutionLevelLock=resolution_level)

Add codes to the dataframe

In [ ]:
df = pd.read_csv(get_files(dstdir, row, "measurements"), index_col=0)
df = pd.DataFrame(df.to_records())
df

In [ ]:
labels = tifffile.imread(get_files(dstdir, row, "labels"))

df = pd.read_csv(get_files(dstdir, row, "measurements"), index_col=0)
df = pd.DataFrame(df.to_records())

channels = get_measurement_channels(df)

Map the intensity on the labels

In [ ]:
cell_intensity = map_intensity(labels, df, channels)

Visualize the map of intensity to select threshold

In [ ]:
# default thresholds
thresholds = {c: df[c].median() + 0.5 * madstd(df[c]) for c in channels}
# custom threshold
thresholds = {"tbh": 0.5, "LGC007": 0.1, "vacht": 0.1}
fig, ax = plt.subplots(1, len(channels))
for k, ch in enumerate(channels):
    ax[k].imshow(
        cell_intensity[k, cell_intensity.shape[1] // 2] > thresholds[ch], cmap="gray"
    )
    ax[k].set_axis_off()
    ax[k].set_title(f"{ch}>{thresholds[ch]:.2f}")

Encode the combination of channels

In [ ]:
codes_in, decoder_in = encode_channels_inclusive(df, channels, thresholds)
codes_ex, decoder_ex = encode_channels_exclusive(df, channels, thresholds)

Show the cells as points for a quick visualization

In [ ]:
fig, ax = plt.subplots()
scatter = ax.scatter(df["x"], df["y"], c=codes_ex["id"], cmap="Set1", alpha=0.25)
ax.axis("equal")
ax.set_title("Exclusive categories")
ax.invert_yaxis()
plt.axis("off")
fig.legend(
    scatter.legend_elements()[0],
    ["+".join(decoder_ex[k]) for k in range(8)],
    loc="outside right",
    title="Catergories",
)

Compute the label code map for the exclusive labels (one single label map for encodes all the categories)

In [ ]:
codemaps, features = create_class_image(labels, codes_ex, decoder_ex)


In [ ]:
plt.imshow(codemaps.max()-codemaps[codemaps.shape[0]//2],cmap='Set1',interpolation=None)
plt.axis('off')
plt.title('Exclusive labels')

Compute the label maps of inclusive labels (one binary map per category)

In [ ]:
maps, names = create_class_masks(labels, codes_in, decoder_in)

Visualization of each inclusive labels

In [ ]:
fig, ax = plt.subplots(1, len(names), figsize=(20, 5))
for k in range(len(names)):
    ax[k].imshow(np.amax(maps[k, :, ::4, ::4], 0), cmap="gray")
    ax[k].set(title=names[k])
    ax[k].title.set_fontsize(5)
    ax[k].set_axis_off()

Visualize the result in 3D using napari
- toggle the label layer to visualize the codes
- on the codemaps layer, tick the 'show selected' option and run through the labels to display the cells code by code

In [ ]:
v = napari.view_image(
    img, channel_axis=1, name=[row[f"channel{k+1}"] for k in range(img.shape[1])]
)
# v.add_labels(labels) # labels is exclusive (categories)
# v.add_image(maps,channel_axis=0,name=names) # maps are inclusive

# Statistical analysis and figure



## Inclusive analysis

Report the percentable of cell positive for each combination of channel if the cell is positive for any of the channels.


In [ ]:
# table with fraction of cells vs  total number of cell in the region
tbl_in = (
    codes_in.merge(df[["label", "roi"]], on="label")
    .drop(columns=["label"])
    .groupby("roi")  # group per roi
    .agg(["count", "sum"])  # count cell and positive cells
    .groupby(axis=1, level=0)  # group per category
    .apply(lambda df: 100 * df[df.columns[1]] / df[df.columns[0]])
)  # ratio
tbl_in

In [ ]:
import seaborn as sns

sns.heatmap(tbl_in.T)
plt.title("Percentage of cell per ROI per category (inclusive)")

Exclusive analysis

In [ ]:
tbl_ex = (
    codes_ex.merge(df[["label", "roi"]], on="label")
    .groupby(["roi", "id"])
    .agg("count")
    .reset_index()
)
tbl_ex["category"] = ["+".join(decoder_ex[x]) for x in tbl_ex["id"]]
tbl_ex = tbl_ex.pivot_table(values="label", columns="category", index="roi")
tbl_ex = tbl_ex.apply(lambda x: 100 * x / x.sum(), axis=1)

In [ ]:
sns.heatmap(tbl_ex.T)
plt.title("Percentage of cell per ROI per category (exclusive)")